In [99]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'dkRewe8bWqDvG_wGkUpZZWfiK76sgOynJi2_lGtlmIOW',
    'service_id': 'iam-ServiceId-f944cf31-4f89-489c-87b7-78cddfcaf29c',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token'}

configuration_name = 'os_8c14de1d42f947c2812d9d92c44fc593_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('winequality-red.csv', 'default-donotdelete-pr-ijbvuypsfmigjt'))
df.take(5)


[Row(fixed acidity='7.4', volatile acidity='0.7', citric acid='0.0', residual sugar='1.9', chlorides='0.076', free sulfur dioxide='11.0', total sulfur dioxide='34.0', density='0.9978', pH='3.51', sulphates='0.56', alcohol='9.4', quality='5'),
 Row(fixed acidity='7.8', volatile acidity='0.88', citric acid='0.0', residual sugar='2.6', chlorides='0.098', free sulfur dioxide='25.0', total sulfur dioxide='67.0', density='0.9968', pH='3.2', sulphates='0.68', alcohol='9.8', quality='5'),
 Row(fixed acidity='7.8', volatile acidity='0.76', citric acid='0.04', residual sugar='2.3', chlorides='0.092', free sulfur dioxide='15.0', total sulfur dioxide='54.0', density='0.997', pH='3.26', sulphates='0.65', alcohol='9.8', quality='5'),
 Row(fixed acidity='11.2', volatile acidity='0.28', citric acid='0.56', residual sugar='1.9', chlorides='0.075', free sulfur dioxide='17.0', total sulfur dioxide='60.0', density='0.998', pH='3.16', sulphates='0.58', alcohol='9.8', quality='6'),
 Row(fixed acidity='7.4',

In [100]:
df.createOrReplaceTempView("df")
df = spark.sql("SELECT cast(`fixed acidity` as float) `fixed acidity`, cast(`volatile acidity` as float) `volatile acidity`,\
          cast(`citric acid` as float) `citric acid`, cast(`residual sugar` as float) `residual sugar`, \
          cast(`chlorides` as float) `chlorides`, cast(`free sulfur dioxide` as float) `free sulfur dioxide`, \
          cast (`total sulfur dioxide` as float) `total sulfur dioxide`, \
          cast(`sulphates` as float) `sulphates`, cast(`alcohol` as float) `alcohol`, cast(`quality` as float) `quality` from df")
df.show(30)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0|     0.56|    9.4|    5.0|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0|     0.68|    9.8|    5.0|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|     0.65|    9.8|    5.0|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|                60.0|     0.58|    9.8|    6.0|
|          7.4|             0.7|        0

In [101]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", 
                                            "free sulfur dioxide", "total sulfur dioxide", "sulphates", "alcohol"],
                                  outputCol="features")


In [102]:
wine_df = vectorAssembler.transform(df)
wine_df = wine_df.select(['features', 'quality'])
wine_df.show(3)

+--------------------+-------+
|            features|quality|
+--------------------+-------+
|[7.40000009536743...|    5.0|
|[7.80000019073486...|    5.0|
|[7.80000019073486...|    5.0|
+--------------------+-------+
only showing top 3 rows



In [105]:
splits = wine_df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [106]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='quality', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(df_train)

print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,-0.246562332057,0.0,0.0,0.0,0.0,0.0,0.0,0.130069722145]
Intercept: 4.426144466062683


In [107]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.736556
r2: 0.176016


In [108]:
df_train.describe().show()

+-------+------------------+
|summary|           quality|
+-------+------------------+
|  count|              1258|
|   mean| 5.655007949125596|
| stddev|0.8117442693894692|
|    min|               3.0|
|    max|               8.0|
+-------+------------------+



In [109]:
lr_predictions = lr_model.transform(df_test)
lr_predictions.select("prediction","quality","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="quality",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-----------------+-------+--------------------+
|       prediction|quality|            features|
+-----------------+-------+--------------------+
|5.869559864798003|    6.0|[5.0,0.7400000095...|
|5.665521454793363|    5.0|[5.0,1.0399999618...|
| 6.09153656061749|    7.0|[5.09999990463256...|
|5.959804873149034|    7.0|[5.09999990463256...|
|5.775795584293244|    5.0|[5.59999990463256...|
+-----------------+-------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.135861


In [110]:
test_result = lr_model.evaluate(df_test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 0.732558


In [111]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 11
objectiveHistory: [0.5, 0.4939489385364445, 0.4806395229195412, 0.4803215112557405, 0.4803087177792582, 0.4803065855778588, 0.4803065711456734, 0.4803065701451617, 0.4803065700720213, 0.48030657006636646, 0.48030657006592137]
+--------------------+
|           residuals|
+--------------------+
|  -2.001845467809119|
| 0.04389357300097174|
|  0.8564356001422979|
|-0.15342689108248742|
|  -0.096467802555817|
|   1.856435600142298|
|  -1.540382974587006|
| 0.02484736144916777|
|  1.0217029548859768|
| -0.5438859404904122|
|-0.16328938230727275|
|-0.16328938230727275|
|  1.1053548653228038|
| 0.10782049364009794|
|  0.1070167066024208|
|  0.9251272742166838|
|  0.9207915589245896|
|  1.0885245346374326|
| -0.6806193487001444|
|  1.0775541029794313|
+--------------------+
only showing top 20 rows



In [112]:
predictions = lr_model.transform(df_test)
predictions.select("prediction","quality","features").show()

+-----------------+-------+--------------------+
|       prediction|quality|            features|
+-----------------+-------+--------------------+
|5.869559864798003|    6.0|[5.0,0.7400000095...|
|5.665521454793363|    5.0|[5.0,1.0399999618...|
| 6.09153656061749|    7.0|[5.09999990463256...|
|5.959804873149034|    7.0|[5.09999990463256...|
|5.775795584293244|    5.0|[5.59999990463256...|
|5.954873616514447|    5.0|[5.59999990463256...|
|5.586980062290905|    4.0|[5.59999990463256...|
|5.928305788699809|    7.0|[5.59999990463256...|
|5.683334709772041|    5.0|[5.59999990463256...|
| 5.86505764630658|    6.0|[5.69999980926513...|
|5.422212333841235|    4.0|[5.69999980926513...|
|5.672918317700854|    6.0|[5.80000019073486...|
|5.719515405404755|    6.0|[5.90000009536743...|
|5.931200447812138|    6.0|[6.0,0.4900000095...|
|5.603560521481262|    5.0|[6.0,0.5,0.039999...|
|5.909009844393404|    6.0|[6.0,0.5799999833...|
| 5.66094876484712|    5.0|[6.09999990463256...|
|5.725983625919463| 